In [9]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50

In [4]:
def load_train(path):
    labels = pd.read_csv(path+'/labels.csv')
    
    train_datagen = ImageDataGenerator(rescale=1./255,
                                      validation_split = 0.25,
                                      rotation_range=45,
                                      horizontal_flip=True)
    
    train_datagen_flow = train_datagen.flow_from_dataframe(dataframe=labels,
                                                           directory = path+'/final_files',
                                                           x_col='file_name',
                                                           y_col='real_age',
                                                           target_size=(224, 224),
                                                           batch_size=32,
                                                           class_mode='raw',
                                                           subset='training',
                                                           seed=42)
    return train_datagen_flow 

In [5]:
def load_test(path):
    labels = pd.read_csv(path+'/labels.csv')
    
    test_datagen = ImageDataGenerator(rescale=1./255,
                                      validation_split = 0.25)
    
    test_datagen_flow = test_datagen.flow_from_dataframe(dataframe=labels,
                                                         directory = path+'/final_files',
                                                         x_col='file_name',
                                                         y_col='real_age',
                                                         target_size=(224, 224),
                                                         batch_size=32,
                                                         class_mode='raw',
                                                         subset='validation',
                                                         seed=42)
    return test_datagen_flow

In [6]:
def create_model(input_shape):
    optimizer = Adam(lr=0.00001)
    
    backbone = ResNet50(input_shape=input_shape,
                        weights='imagenet',
                        include_top=False)     
    
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='relu')) 
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics=['mae'])
    return model

In [7]:
def train_model(model, train_data,
                test_data, batch_size=None, epochs=50,
                steps_per_epoch=None, validation_steps=None):
    
    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)
        
    model.fit(train_data,    
              validation_data=test_data,
              batch_size=batch_size, 
              epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2)
    return model 